# The Dice Game
## Preamble

Code bellow represents an agent that can play a simple dice game. Here are the basic rules:
* You start with 0 points
* Roll three fair six-sided dice
* Now choose one of the following:
 * Stick, accept the values shown. If two or more dice show the same values, then all of them are flipped upside down: 1 becomes 6, 2 becomes 5, 3 becomes 4, and vice versa. The total is then added to your points and this is your final score.
 * OR reroll the dice. You may choose to hold any combination of the dice on the current value shown. Rerolling costs you 1 point – so during the game and perhaps even at the end your score may be negative. You then make this same choice again.

The best possible score for this game is 18 and is achieved by rolling three 1s on the first roll.

The reroll penalty prevents you from rolling forever to get this score. If the value of the current dice is greater than the expected value of rerolling them (accounting for the penalty), then you should stick.

The optimal decision is independent of your current score. It does not matter whether it is your first roll with a current score of 0, or your twentieth roll with a current score of -19 (in which case a positive end score is impossible), in either of these cases if you roll three 6s (which, if you stick, will only add 3 points) then you still expect to get a *better* end score by rerolling and taking the penalty. Almost any other roll will beat it, so it's still the right choice to maximise your score.

It is pretty obvious that you should stick on three 1s, and reroll on three 6s. Should you hold any of the 6s when you reroll? What about other values? What should you do if the dice come up 3, 4, 5?

We do not know what numbers will come up when we roll, but we do know exactly what the probability of any given roll is. This is the point of the probabilistic reasoning section of the unit; if we can model the true probabilities then we can mathematically calculate the optimal policy. Not all real world situations use dice, but these techniques work well even if we can only estimate the true probabilities.

### Play The Game
You can play the game in the following cell. Change the `SKIP_GAME` constant to `False` to enable this cell. 
<br> *Make sure to change it back to `True` before submitting.*

In [1]:
SKIP_GAME = True
if not SKIP_GAME:
    %run dice_game.py

The code supports playing this game with many possible modifications – you can change the number of dice, the values on the dice, or even make biased (weighted) dice that are more likely to roll certain values. More on this later.
 
### Choice of Algorithm
*value iteration* technique will work well here to produce a strategy for the game, which the agent can then follow. Working with the parameters that will be suit the value iteration algorithm to maximise your score is an option.

However, there are many other possible options. Simply calculating the expected value of a single roll will produce a much stronger strategy than playing randomly. You could also look up various other approaches that can be applied to Markov decision processes, such as policy iteration.

A great agent will require a particularly efficient implementation value iteration with intelligent choice of parameters.


## Dice Game Class
A class called `DiceGame` is provided within `dice_game.py` 
When a DiceGame object is created, by default you will get the rules as stated above. 
```python
game = DiceGame()
```
Creates a game with 3 normal 6-sided dice. 

Game mechanics can be modified, and can be achieved by using the other constructor arguments, for example:
```python
game = DiceGame(dice=4, sides=3, values=[1, 2, 6], bias=[0.1, 0.1, 0.8], penalty=2)
```
will create a game where you roll 4 dice, each with 3 sides, labelled 1, 2, and 6, where each die is far more likely to roll a 6 than they are to roll a 1 or a 2, and furthermore the penalty for rerolling is now 2 points instead of 1. *Note: this does not necessarily result in an interesting game.*

In games with unusual values or sides (3-sided dice are unusual without trying to turn them upside down), when there are duplicates, `value[i]` becomes `value[-i]`. With odd-sided dice, the middle value will flip onto itself.

Once created, the `DiceGame` object can be run in two different modes, *simulation* and *analysis*. It is likely that you will mostly use [*analysis* mode](#Analysis-Mode) to derive your agent's behaviour, but either way some understanding of simulation mode might be useful.

### Simulation Mode
The object provides the methods required to simulate playing the game. This might be useful for trsting purposes. The current dice values are found by calling `get_dice_state()`, they will always be listed in ascending order. 

In [2]:
from dice_game import DiceGame
import numpy as np

# setting a seed for the random number generator gives repeatable results, making testing easier!
np.random.seed(111)

game = DiceGame()
game.get_dice_state()

(2, 3, 4)

To roll the dice, you call the `roll` method which takes one parameter: a tuple representing which dice you want to hold, numbered from zero. We rolled (2, 3, 4). Suppose we want to hold the 2, we would pass the tuple `(0,)` into the `roll` method (note we need to include the comma so that Python knows this is a tuple).

The `roll` method returns a tuple containing: the reward for this action, the new state, and whether the game is over.

In [3]:
reward, new_state, game_over = game.roll((0,))
print(reward)
print(new_state)
print(game_over)

-1
(2, 2, 5)
False


Now suppose we are happy and wish to stick to get our final score. We can call the `roll` method and supply a tuple containing all three dice.

In [4]:
reward, new_state, game_over = game.roll((0, 1, 2))
print(reward)
print(new_state)
print(game_over)

15
(5, 5, 5)
True


Remember that the return value is just the reward for the action, in this case 15. To get our final score we can inspect `game.score`. We rerolled once, so expect to get a score of 14.

In [5]:
print(game.score)

14


And to play again we can call `game.reset()` which returns the new starting dice.

In [6]:
game.reset()

(1, 1, 3)

### Analysis Mode
In analysis mode, you are not playing the game, but asking the object for *all possible outcomes of certain actions*.

First of all, it is useful to know that all the possible states and all the possible actions are stored inside the object. Try changing the game mechanics on the first line (e.g. add `dice=2` to the constructor) and see how the other information is updated.

In [7]:
game = DiceGame()
print(f"The first 5 of {len(game.states)} possible dice rolls are: {game.states[0:5]}")
print(f"The possible actions on any given turn are: {game.actions}")

The first 5 of 56 possible dice rolls are: [(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 1, 5)]
The possible actions on any given turn are: [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]


Finally, the most important method is `get_next_states(action, dice_state)`. This allows you to get all the possible resulting states for any given state and action.

Earlier we had the roll of `(2, 3, 4)` and decided to hold the 2. The game can calculate all possible outcomes for us, and crucially will also give us the probability of each state occurring.

In [8]:
game = DiceGame()
states, game_over, reward, probabilities = game.get_next_states((0,), (2, 3, 4))
for state, probability in zip(states, probabilities):
    print(f"Would get roll of {state} with probability {probability}")

Would get roll of (1, 1, 2) with probability 0.02777777777777778
Would get roll of (1, 2, 2) with probability 0.055555555555555566
Would get roll of (1, 2, 3) with probability 0.055555555555555566
Would get roll of (1, 2, 4) with probability 0.055555555555555566
Would get roll of (1, 2, 5) with probability 0.055555555555555566
Would get roll of (1, 2, 6) with probability 0.05555555555555559
Would get roll of (2, 2, 2) with probability 0.02777777777777778
Would get roll of (2, 2, 3) with probability 0.055555555555555566
Would get roll of (2, 2, 4) with probability 0.055555555555555566
Would get roll of (2, 2, 5) with probability 0.055555555555555566
Would get roll of (2, 2, 6) with probability 0.05555555555555559
Would get roll of (2, 3, 3) with probability 0.02777777777777778
Would get roll of (2, 3, 4) with probability 0.055555555555555566
Would get roll of (2, 3, 5) with probability 0.055555555555555566
Would get roll of (2, 3, 6) with probability 0.05555555555555559
Would get roll o

The method also works consistently when all dice are held, reporting that this action would cause the game to be over and giving the reward. Note that the list of states returned contains the value `None`. This is to denote that the game has entered a terminal state – no further actions would be allowed. The game does not return the final dice here, because that is another valid state (from which there would still be actions available). Also, the `reward` value is not the same as the final `score` of any given game, because it does not include any possible previous penalties for rerolling.

In [9]:
states, game_over, reward, probabilities = game.get_next_states((0, 1, 2), (2, 2, 5))
print(states)
print(game_over)
print(reward)

[None]
True
15


## Part One

Let's start with some example agents, so you can see the format we will use. In the cell below are two agents which do not play particularly well. One always holds immediately, the other will keep re-rolling all dice until they get the best possible dice (`(1, 1, 1)` or `(1, 1, 6)`), ignoring the massive penalty this will incur from re-rolling. Neither of them is considering the probabilities involved in the game.

There is also a function which will run the game with an instance of a given agent. When you run the cell, it will simulate a game with each agent.

In [10]:
# reset all the variables from previous cells
%reset -f

In [11]:
from abc import ABC, abstractmethod
from dice_game import DiceGame
import numpy as np


class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass


class AlwaysHoldAgent(DiceGameAgent):
    def play(self, state):
        return (0, 1, 2)


class PerfectionistAgent(DiceGameAgent):
    def play(self, state):
        if state == (1, 1, 1) or state == (1, 1, 6):
            return (0, 1, 2)
        else:
            return ()
        
        
def play_game_with_agent(agent, game, verbose=False):
    state = game.reset()
    
    if(verbose): print(f"Testing agent: \n\t{type(agent).__name__}")
    if(verbose): print(f"Starting dice: \n\t{state}\n")
    
    game_over = False
    actions = 0
    while not game_over:
        action = agent.play(state)
        actions += 1
        
        if(verbose): print(f"Action {actions}: \t{action}")
        _, state, game_over = game.roll(action)
        if(verbose and not game_over): print(f"Dice: \t\t{state}")

    if(verbose): print(f"\nFinal dice: {state}, score: {game.score}")
        
    return game.score


def main():
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    #np.random.seed(1)
    
    game = DiceGame()
    
    agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")
    
    agent2 = PerfectionistAgent(game)
    play_game_with_agent(agent2, game, verbose=True)
    

if __name__ == "__main__":
    main()

Testing agent: 
	AlwaysHoldAgent
Starting dice: 
	(1, 1, 6)

Action 1: 	(0, 1, 2)

Final dice: (6, 6, 6), score: 18


Testing agent: 
	PerfectionistAgent
Starting dice: 
	(5, 5, 6)

Action 1: 	()
Dice: 		(4, 5, 6)
Action 2: 	()
Dice: 		(1, 3, 3)
Action 3: 	()
Dice: 		(1, 5, 5)
Action 4: 	()
Dice: 		(2, 4, 6)
Action 5: 	()
Dice: 		(1, 3, 4)
Action 6: 	()
Dice: 		(2, 3, 5)
Action 7: 	()
Dice: 		(2, 3, 6)
Action 8: 	()
Dice: 		(1, 5, 6)
Action 9: 	()
Dice: 		(5, 6, 6)
Action 10: 	()
Dice: 		(3, 3, 6)
Action 11: 	()
Dice: 		(1, 1, 2)
Action 12: 	()
Dice: 		(1, 1, 1)
Action 13: 	(0, 1, 2)

Final dice: (6, 6, 6), score: 6


Agent implementation

In [12]:
from dice_game import *
import time

class MyAgent(DiceGameAgent):
    
    #__slots__ = 'game' # Reduce the size of object
    
    #####################################################
    ################   Constructor  #####################
    #####################################################
    # Value iteration algorithm inside
    
    def __init__(self, game): 
    
        super().__init__(game)
        #print(sys.getsizeof(game))
        
        # Paramaters and varaibles:
        # self.gamma = discount factor-> Optimal bteween 0.90 an 1 "Modify if it´s needed"
        # converged-> Flag to the "while loop-Value_iteration algorithm" for self.convergence_criterion "Do NOT Modify"
        # self.convergence_criterion-> Defining a treshold to limit the value_iteration function "Modify if it´s needed"
        #                              Estimated average runs: 6 (2.3s ->1e-9), (1.4s->1e-3), (1.6s->1e-4)
        # self.memory_backup-> Save states and actions that agent has already explored 
        # self.policy-> Save all States with their action
        # self.values_s-> Save all states eith their values
        
        converged = False
        convergence_criterion = 1e-3
        self.gamma = 0.95
        self.memory_backup = {}
        self.policy = {}
        self.values_s = {}
        
        # Getting all states and values based on "game" object (baseline dice=3, sides=6)
        
        for state in game.states:
            self.values_s[state] = game.final_score(state)
        
        #####################################################
        ##########   Value_Iteration Algorithm  #############
        #####################################################
        
        while not converged:
            
            convergence_factor = 0 # Initialazing the convergence_factor for each iteration
            
            for state in game.states: # self.values_s.keys() could be used but time does NOT decrease
                
                new_value = float("-inf") # Initialazing the value with the highest negative number for each state
                old_value = self.values_s[state] # Save the state´s value
                                
                for action in game.actions:
                    
                    Bellman_value = self.Bellman_value_function(action, state)

                    if Bellman_value > new_value: # Updating the Bellman´s value (maximazing value) 
                        new_value = Bellman_value 
                        
                        optimal_action = action # Extracting the induced policy / action to keep
                        #print(f"optimal_action->{optimal_action}")
                
                self.values_s[state] = new_value # Assigning the updated value to the corresponding state
                self.policy[state] = optimal_action # Assigning the induced policy according to the best last value
                
            # convergence method based on self.convergence_criterion
                convergence_factor = max(convergence_factor, abs(old_value - new_value))
            if convergence_factor < convergence_criterion:
                break
                
        return None
            
    
    #####################################################
    ##########   Bellman_value_function   ###############
    #####################################################
    # Getting the Bellman´s value
    
    def Bellman_value_function(self, action, state, V_s = 0):
        
        # Function to reduce process time
        self.states_prob, self.game_over, self.reward = self.memory_backup_function(action, state) 
        
        for next_state, probability in self.states_prob:
            
            if not self.game_over: # Iterating until True
                V_s += probability * (self.reward + self.gamma * self.values_s.get(next_state))
            else: # End game (end_game = True)
                V_s += probability * (self.reward + self.gamma * self.game.final_score(state))
    
        return V_s
    
    
    #####################################################
    ##########   memory_backup_function   ###############
    #####################################################
    # Save actions and sates (based on get_next_states) to self.memory_backup in order to speed up
    # the calculation and search process (- time but + memory)
    # Return a dictionary with states and their probabilities (self.states_prob)
    # Return the status of game over (flag) and the reward

    def memory_backup_function(self, action, state):
        
        if (action, state) not in self.memory_backup:
            self.memory_backup[(action, state)] = self.game.get_next_states(action, state)
   
        self.other_states, self.game_over, self.reward, self.probabilities = self.memory_backup.get((action, state))
        
        self.states_prob = zip(self.other_states, self.probabilities)
        
        return self.states_prob, self.game_over, self.reward
    
    
    #####################################################
    ###################   play   ########################
    #####################################################
    # Return the action based on the state according to updated best last value in Bellman´s equation
    
    def play(self, state):
        action = self.policy.get(state)
        return action

#def main():
    
    #game = DiceGame()
    
    #My_agent = MyAgent(game)
    #play_game_with_agent(My_agent, game, verbose=True)
    

#if __name__ == "__main__":
    #total_time = 0
    #start_time = time.process_time()
    #main()
    #total_time += time.process_time() - start_time

    #print(f"Total time: {total_time:.4f} seconds")


### Testing Details
Agent will be tested in two ways. First it will be tested in actual random games, and the average score will be measured. All students will get the exact same dice rolls, so the best strategies will get the most points. Second, it will be analysed in specific circumstances (i.e. specific dice rolls) to test what it does compared to the optimal strategy.

In addition, the tests will be repeated with *extended rules*, i.e. not using the default game with three 6-sided fair dice. You can read more about how this affects grading in [this section](#Choice-of-Algorithm). 

On the testing machine, the agent must take less than 30 seconds to construct, and less than 2 seconds to produce each action.

The best way to improve the performance is through a detailed understanding and smart choice of AI algorithms. This implementaion is ***not*** meant to test the ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
Run the following cell to test the agent 10 times and print the average score.

To enable the tests, set the constant `SKIP_TESTS` to `False`.

In [13]:
SKIP_TESTS = True

def tests():
    import time

    total_score = 0
    total_time = 0
    n = 10

    np.random.seed()

    print("Testing basic rules.")
    print()

    game = DiceGame()

    start_time = time.process_time()
    test_agent = MyAgent(game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Total time: {total_time:.4f} seconds")
    
if not SKIP_TESTS:
    tests()

#### Opt-In For Extended Rules
Now, the agent will be tested with rules other than the defaults.
Set the constant `TEST_EXTENDED_RULES` to `True` on the next line. Another test will be performed to check if the code still works. If an error occuts, it would indicate that the code is not supporting the extended rules properly.

Refer to [this section](#Choice-of-Algorithm) to understand more about how extended rules factor into your possible grade.

**Note:** you need to have `SKIP_TESTS` set to `False` in the cell above (and run it!) to enable the tests below.

In [14]:
TEST_EXTENDED_RULES = True

def extended_tests():
    total_score = 0
    total_time = 0
    n = 10

    print("Testing extended rules – two three-sided dice.")
    print()

    game = DiceGame(dice=2, sides=3)

    start_time = time.process_time()
    test_agent = MyAgent(game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Average time: {total_time/n:.5f} seconds")

if not SKIP_TESTS and TEST_EXTENDED_RULES:
    extended_tests()

## Checking code
The following cell tests if your notebook is ready

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then it is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

In [15]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the earlier cell.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./dicegame.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named dicegame.ipynb")

    if "MyAgent" not in globals():
        fail = True;
        print("You must include a class called MyAgent as defined above.")
    else:    
        game = DiceGame()
        agent = MyAgent(game)
        action = agent.play((1, 1, 1))

        if action not in game.actions:
            print("Warning:")
            print("Your agent does not seem to produce a valid action with the default rules.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

        if TEST_EXTENDED_RULES:
            print("Info: TEST_EXTENDED_RULES is ON")
            game = DiceGame(dice=2, sides=8)
            agent = MyAgent(game)
            try:
                action = agent.play((7, 8))
            except:
                action = None

            if action not in game.actions:
                fail = True
                print("Your agent does not produce a valid action with the extended rules.")
                print("Turn off TEST_EXTENDED_RULES if you cannot fix this error.")
        else:
            print("Info: TEST_EXTENDED_RULES is OFF (extended rules will not be tested)")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print()
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

Info: TEST_EXTENDED_RULES is ON

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [16]:
# This is a TEST CELL. Do not delete or change.